In [61]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,mean_absolute_percentage_error
import catboost

In [62]:
DATA_PATH = '../../data/'
X_train = pd.read_csv(f'{DATA_PATH}X_train.csv',sep='\t', index_col='index')
X_test = pd.read_csv(f'{DATA_PATH}X_test.csv', sep='\t', index_col='index')
y_train = pd.read_csv(f'{DATA_PATH}y_train.csv',sep='\t', index_col='index')
y_test = pd.read_csv(f'{DATA_PATH}y_test.csv',sep='\t', index_col='index')

In [63]:
target_columns = y_train.columns

In [64]:
models = {}
for target in target_columns:
    eval_data = catboost.Pool(X_test,y_test[target])
    model = catboost.CatBoostRegressor(iterations=10000,
                                learning_rate=0.01,
                                task_type='CPU',
                                loss_function='MAE',
                                random_seed= 42)
    model.fit(X_train,y_train[target],use_best_model=True, early_stopping_rounds=1000, eval_set=eval_data, silent=True)
    models[target] = model

In [65]:
r2_scores = np.zeros(4)
mae_scores = np.zeros(4)
mape_scores = np.zeros(4)
mse_scores = np.zeros(4)
for itarget, target in enumerate(target_columns):
    r2_scores[itarget] = r2_score(y_test[target],models[target].predict(X_test))
    mse_scores[itarget] = mean_squared_error(y_test[target],models[target].predict(X_test))
    mape_scores[itarget] = mean_absolute_percentage_error(y_test[target],models[target].predict(X_test))
    mae_scores[itarget] = mean_absolute_error(y_test[target],models[target].predict(X_test))

In [66]:
scores = pd.DataFrame(index=target_columns,columns=['R2','MAE','MSE','MAPe'])

In [67]:
scores['R2'] = r2_scores
scores['MAE'] = mae_scores
scores['MSE'] = mse_scores
scores['MAPe'] = mape_scores

In [68]:
scores




,R2,MAE,MSE,MAPe
химшлак последний Al2O3,0.417439,0.508752,0.568331,0.109536
химшлак последний CaO,0.209459,2.536374,10.814222,0.046224
химшлак последний R,0.398723,0.136182,0.030502,0.057322
химшлак последний SiO2,0.203819,1.524374,3.774773,0.065098
